In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne

/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
from OpenData import unpickle

In [3]:
data_array = []
labels_array = []

In [4]:
for i in range(5):
    tmp = unpickle('cifar-10/data_batch_' + str(i + 1))
    data_array.append(tmp['data'])
    labels_array.append(tmp['labels'])

In [5]:
tmp = unpickle('cifar-10/test_batch')
data_array.append(tmp['data'])
labels_array.append(tmp['labels'])

In [6]:
data_normalized = np.concatenate(data_array)/np.float32(255)
data = np.dstack((data_normalized[:, :1024], data_normalized[:, 1024:2048], data_normalized[:, 2048:]))
data = data.reshape((data.shape[0], 32, 32, 3)).transpose(0,3,1,2)

In [7]:
labels = np.concatenate(labels_array)

In [8]:
test_indexes = np.random.randint(0,len(labels), 60000)
train_indexes = list(set(range(len(labels))) - set(test_indexes))

In [9]:
data_train = data[train_indexes]
labels_train = labels[train_indexes]

data_test = data[test_indexes]
labels_test = labels[test_indexes]

In [10]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [11]:
def build_cnn(input_var=None):
    network = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_var)
    
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=64, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))
    
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=128,
            nonlinearity=lasagne.nonlinearities.rectify)

    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10, #classes
            nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [12]:
#### Далее -- копипаста.

In [13]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

In [14]:
network = build_cnn(input_var)

In [15]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

/usr/local/lib/python2.7/dist-packages/lasagne/layers/conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


In [16]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.01, momentum=0.9)

test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
test_loss = test_loss.mean()

test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                        dtype=theano.config.floatX)

In [17]:
pred = test_prediction.argmax(-1)
f_predict = theano.function([input_var], pred)

In [18]:
train_fn = theano.function([input_var, target_var], 
                           loss, 
                           updates=updates,
                           allow_input_downcast=True)

val_fn = theano.function([input_var, target_var], 
                         [test_loss, test_acc],
                         allow_input_downcast=True)

In [19]:
num_epochs = 2

In [20]:
BATCH_SIZE = 512 #60000 / 100

In [21]:
import time

In [22]:
for epoch in range(num_epochs):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(data_train, labels_train, BATCH_SIZE, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(data_test, labels_test, BATCH_SIZE, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1
        
    print("Epoch: {}, train:{}, val:{}, time:{}".format(epoch, 
                                               train_err/train_batches,
                                               val_err/val_batches,
                                               time.time() - start_time))

Epoch: 0, train:2.23219909593, val:2.073441811, time:189.219553947
Epoch: 1, train:2.04347846635, val:1.92403068764, time:180.588929892


In [23]:
y_true = []
y_pred = []

start_time = time.time()
for i in range(len(labels_test)):
    y_true.append(int(labels_test[i]))
    y_pred.append(f_predict([data_test[i]])[0])
    
print("time:{}".format(time.time() - start_time))

time:289.3807199


In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
CM = confusion_matrix(y_true, y_pred)

In [27]:
import seaborn as sns

fig = plt.figure(figsize=(10, 10))
sns.heatmap(CM, annot=True, fmt="d", linewidths=.5)
plt.show()

NameError: name 'plt' is not defined